In [3]:
import pandas as pd
import numpy as np

In [4]:
pd.set_option('display.max_rows', None)

In [34]:
OUTPUT_DIR="output"
OUTPUT_TIME="time"

EXECUTIONS = 50
CITIES = ["13", "14", "15"]
THREADS = ["1", "2", "3", "6", "9", "12"]
ITERATIONS = 10

In [35]:
def extract_time_stats_from_file(file_name):
    file = open(file_name)
    data = []
    for exec in range(1, EXECUTIONS + 1):
        data.append(float(file.readline().split(" ")[1]))
    np_array = np.array(data)
    file.close()
    return np.mean(np_array), np.std(np_array), np.sum(np_array)


In [36]:
df_time = pd.DataFrame(columns = ["Version", "Cities", "Threads", "Instance_mean", "Instance_std", "Sum_mean", "Sum_std"])

for city in list(CITIES):
    mean_array, std_array, sum_array = [], [], []
    for iteration in range(1,ITERATIONS + 1):
        file_name = OUTPUT_DIR + "/" + str(iteration) + "_seq_" + city + "_cities_" + str(EXECUTIONS) + "_executions_result.out"
        
        mean, std, sum = extract_time_stats_from_file(file_name)
        
        mean_array.append(mean)
        std_array.append(std)
        sum_array.append(sum)
    # print(mean_array, std_array)
    list_row = ["Sequential", city, "1", np.mean(mean_array), np.mean(std_array), np.mean(sum_array), np.std(sum_array)]
    df_time.loc[len(df_time)] = list_row
            
    for thread in list(THREADS):
        mean_array, std_array, sum_array = [], [], []
        for iteration in range(1,ITERATIONS + 1):
            file_name = OUTPUT_DIR + "/" + str(iteration) + "_par_" + city + "_cities_" + str(EXECUTIONS) + "_executions_" + thread + "_threads_result.out"
            
            mean, std, sum = extract_time_stats_from_file(file_name)
        
            mean_array.append(mean)
            std_array.append(std)
            sum_array.append(sum)
        list_row = ["Parallel", city, thread, np.mean(mean_array), np.mean(std_array), np.mean(sum_array), np.std(sum_array)]
        df_time.loc[len(df_time)] = list_row
df_time

,Version,Cities,Threads,Instance_mean,Instance_std,Sum_mean,Sum_std
0,Sequential,13,1,0.495481,0.397178,24.774059,0.187431
1,Parallel,13,1,0.688790,0.514890,34.439478,0.074784
2,Parallel,13,2,0.278350,0.217408,13.917523,0.018476
3,Parallel,13,3,0.185413,0.144959,9.270669,0.028184
4,Parallel,13,6,0.094327,0.073411,4.716368,0.010189
5,Parallel,13,9,0.076241,0.058210,3.812070,0.010389
6,Parallel,13,12,0.065362,0.048934,3.268090,0.006673
7,Sequential,14,1,2.581947,3.837842,129.097369,0.390816
8,Parallel,14,1,3.704861,4.984478,185.243063,0.248426
9,Parallel,14,2,1.425104,2.078227,71.255180,0.123081


In [51]:
df_speedup_efficiency = pd.DataFrame(columns = ["Cities", "Threads", "Sum Speedup", "Sum Efficiency"])

df_sequential = df_time[df_time['Version'].isin(["Sequential"])]
df_parallel = df_time[df_time['Version'].isin(["Parallel"])]

for city in list(CITIES):
    df_seq = df_sequential[df_sequential["Cities"].isin([city])]
    for thread in list(THREADS):
        df_par = df_parallel[df_parallel["Cities"].isin([city]) & df_parallel["Threads"].isin([thread])]
        # speedup_instance = df_seq["Instance_mean"].item()/df_par["Instance_mean"].item()
        # efficiency_instance = df_seq["Instance_mean"].item()/df_par["Instance_mean"].item()/float(thread)
        speedup_sum = df_seq["Sum_mean"].item()/df_par["Sum_mean"].item()
        efficiency_sum = df_seq["Sum_mean"].item()/df_par["Sum_mean"].item()/float(thread)

        # list_row = [city, thread, speedup_instance, efficiency_instance, speedup_sum, efficiency_sum]
        list_row = [city, thread, speedup_sum, efficiency_sum]
        df_speedup_efficiency.loc[len(df_speedup_efficiency)] = list_row
df_speedup_efficiency

,Cities,Threads,Sum Speedup,Sum Efficiency
0,13,1,0.719351,0.719351
1,13,2,1.780062,0.890031
2,13,3,2.672305,0.890768
3,13,6,5.252784,0.875464
4,13,9,6.498847,0.722094
5,13,12,7.580593,0.631716
6,14,1,0.696908,0.696908
7,14,2,1.811761,0.905881
8,14,3,2.742675,0.914225
9,14,6,5.387258,0.897876
